# செமாண்டிக் கர்னல் கருவி பயன்பாட்டு உதாரணம்

இந்த ஆவணம் செமாண்டிக் கர்னல் அடிப்படையிலான ஒரு கருவியை உருவாக்க பயன்படுத்தப்படும் குறியீட்டின் மேற்பார்வை மற்றும் விளக்கத்தை வழங்குகிறது, இது Azure AI Search உடன் Retrieval-Augmented Generation (RAG) ஐ ஒருங்கிணைக்கிறது. இந்த உதாரணம் Azure AI Search குறியீட்டு மூலம் பயண ஆவணங்களை பெறும், பயனர் கேள்விகளை செமாண்டிக் தேடல் முடிவுகளுடன் மேம்படுத்தும், மற்றும் விரிவான பயண பரிந்துரைகளை ஸ்ட்ரீம் செய்யும் AI முகவரை உருவாக்குவது எப்படி என்பதை விளக்குகிறது.


## சூழலை ஆரம்பித்தல்


### பாக்கேஜ்களை இறக்குமதி செய்வது  
கீழே உள்ள குறியீடு தேவையான பாக்கேஜ்களை இறக்குமதி செய்கிறது:


In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### செமாந்திக் கர்னல் மற்றும் AI சேவையை உருவாக்குதல்

ஒரு செமாந்திக் கர்னல் உதாரணம் உருவாக்கப்பட்டு, அசிங்கமான OpenAI உரையாடல் நிறைவு சேவையுடன் அமைக்கப்படுகிறது. பதில்களை உருவாக்க பயன்படுத்த சேவை கர்னலுக்கு சேர்க்கப்படுகிறது.


In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### PromptPlugin ஐ வரையறுத்தல்

PromptPlugin என்பது retrieval context பயன்படுத்தி விரிவாக்கப்பட்ட prompt ஒன்றை உருவாக்க ஒரு செயல்பாட்டை வரையறுக்கும் ஒரு இயற்கை பிளகின்.


In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## வெக்டர் தரவுத்தொகுப்பு தொடக்கம்

நாங்கள் Azure AI Search-ஐ நிலையான சேமிப்புடன் தொடங்குகிறோம் மற்றும் மேம்படுத்தப்பட்ட மாதிரித் ஆவணங்களைச் சேர்க்கிறோம். Azure AI Search ஆவணங்களைச் சேமிக்கவும், துல்லியமான பதில்களை உருவாக்க உதவும் சூழலை மீட்டெடுக்கவும் பயன்படுத்தப்படும்.


In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### ஸ்ட்ரீமிங் அழைப்புடன் ஏஜெண்டை இயக்குதல்

முக்கிய அசிங்க்ரோனஸ் லூப் உரையாடலுக்கான ஒரு த்ரெடைப் உருவாக்குகிறது, மேலும் ஒவ்வொரு பயனர் உள்ளீட்டிற்கும், ஏஜெண்ட் மீட்பு சூழலைப் பார்க்கும் வகையில் செயல்படுகிறது. பயனர் செய்தியும் சேர்க்கப்படுகிறது, பின்னர் ஸ்ட்ரீமிங் மூலம் ஏஜெண்ட் அழைக்கப்படுகிறது. வெளியீடு ஸ்ட்ரீமிங் மூலம் வரும்போது அச்சிடப்படுகிறது.


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


நீங்கள் கீழே உள்ளதைப் போன்ற ஒரு வெளியீட்டை காணலாம்:

```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
